# Notebook: LDA

## 1. Load Packages

In [1]:
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
from gensim import corpora
import pandas as pd

## 2. Constants / Setup

In [ ]:
N_TOPICS = 10

In [2]:
FILTERED_DATASET_MENTIONS_PATH = "../Datasets/filtered_mentions.csv"
FILTERED_DATASET_POLITICIANS_PATH = "../Datasets/filtered_politicians.csv"

## 3. Load Dataset

In [3]:
df_mentions = pd.read_csv(FILTERED_DATASET_MENTIONS_PATH)
df_mentions

,Unnamed: 0,text,source_party,source_account,date,sentiment,clean_text
0,0,Wichtige wissenschaftliche Erkenntnis- nun mus...,SPD,KarambaDiaby,2021-01-09 19:35:29,0,wichtige wissenschaftliche erkenntnis schnell ...
1,1,@KarambaDiaby @HalleSpd @SPD_LSA Ich gratulier...,SPD,KarambaDiaby,2021-01-09 17:09:28,0,gratuliere linken
2,2,@KarambaDiaby @HalleSpd @SPD_LSA Herzlichen Gl...,SPD,KarambaDiaby,2021-01-09 13:16:13,0,herzlichen glückwunsch erfolg
3,3,@KarambaDiaby @HalleSpd @SPD_LSA Wann werden k...,SPD,KarambaDiaby,2021-01-09 12:32:40,1,wann konkret massiv steuern sozialabgaben gese...
4,4,@KarambaDiaby @HalleSpd @SPD_LSA Glückwunsch.,SPD,KarambaDiaby,2021-01-09 12:13:06,0,glückwunsch
...,...,...,...,...,...,...,...
707236,707236,@b_riexinger Klima oder Verkehr fast gleich......,LINKE,b_riexinger,2021-12-17 08:19:23,1,klima verkehr fast gleich hauptsache pöstchen
707237,707237,@b_riexinger @Linksfraktion Na ob das noch lan...,LINKE,b_riexinger,2021-12-17 08:18:07,1,na lange gut geht gruppierungen querdenker lin...
707238,707238,@b_riexinger Ich wünsch Dir viel Erfolg.,LINKE,b_riexinger,2021-12-17 07:47:59,0,wünsch erfolg
707239,707239,"@b_riexinger Nun, da gibt es ja genügend zu tu...",LINKE,b_riexinger,2021-12-17 02:07:26,2,gibt ja genügend tuen paris macht


In [4]:
df_politicians = pd.read_csv(FILTERED_DATASET_POLITICIANS_PATH)
df_politicians

,Unnamed: 0,UserScreenName,source_account,date,Text,text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL,id,source_party,sentiment,clean_text
0,0,AfD Berlin,AfDBerlin,2021-03-26 21:07:22,AfD Berlin\n@AfDBerlin\n·\n26. März,AfD wirkt.\n\nSchluss mit dem #Gendergaga\nMDR...,NaN,4.0,28.0,132.0,['https://pbs.twimg.com/profile_images/1037343...,https://twitter.com/AfDBerlin/status/137555499...,1,AfD,2,afd wirkt schluss gendergaga mdr aktuell märz ...
1,1,AfD Berlin,AfDBerlin,2021-03-27 07:20:27,AfD Berlin\n@AfDBerlin\n·\n27. März,Im Herbst wird gewählt.\nSchluss mit den Recht...,NaN,10.0,20.0,112.0,['https://pbs.twimg.com/card_img/1471780757332...,https://twitter.com/AfDBerlin/status/137570928...,2,AfD,2,herbst gewählt schluss rechtsbrüchen regierung...
2,2,AfD Berlin,AfDBerlin,2021-03-31 07:14:04,AfD Berlin\n@AfDBerlin\n·\n31. März,Behördenwillkür\nFlüchtlingsheime durchgedrück...,NaN,3.0,13.0,34.0,['https://pbs.twimg.com/media/ExylKvEU8AgowSU?...,https://twitter.com/AfDBerlin/status/137715723...,3,AfD,1,behördenwillkür flüchtlingsheime durchgedrückt...
3,3,AfD Berlin,AfDBerlin,2021-04-01 14:29:00,AfD Berlin\n@AfDBerlin\n·\n1. Apr.,Aus Raider wird jetzt Twix \n\nLeider kein #Ap...,NaN,1.0,4.0,17.0,['https://pbs.twimg.com/card_img/1472910546907...,https://twitter.com/AfDBerlin/status/137762907...,4,AfD,2,raider twix leider aprilscherz bz berlin neuer...
4,4,AfD Berlin,AfDBerlin,2021-04-01 05:02:10,AfD Berlin\n@AfDBerlin\n·\n1. Apr.,Gendern geht („*innen“)\nImpfen geht nicht.\nD...,NaN,1.0,3.0,16.0,['https://pbs.twimg.com/profile_images/9706413...,https://twitter.com/AfDBerlin/status/137748642...,5,AfD,1,gendern geht innen impfen geht regierung blami...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58859,58859,Tino Chrupalla,Tino_Chrupalla,2021-12-04 17:26:46,Tino Chrupalla\n@Tino_Chrupalla\n·\n4. Dez. 2021,Friedlicher Protest gegen einen #Impfzwang ist...,NaN,265.0,122.0,578.0,[],https://twitter.com/Tino_Chrupalla/status/1467...,61802,AfD,1,friedlicher protest impfzwang grundrecht schut...
58860,58860,Tino Chrupalla,Tino_Chrupalla,2021-12-13 16:30:19,Tino Chrupalla\n@Tino_Chrupalla\n·\n13. Dez. 2021,@OlafScholz\n muss sich endlich klar zu Nord S...,NaN,4.0,2.0,11.0,[],https://twitter.com/Tino_Chrupalla/status/1470...,61803,AfD,0,endlich klar nord stream bekennen deutsche int...
58861,58861,Tino Chrupalla,Tino_Chrupalla,2021-12-17 14:02:24,Tino Chrupalla\n@Tino_Chrupalla\n·\n17. Dez. 2021,Mit \n@_FriedrichMerz\n gibt es keine konserva...,NaN,188.0,227.0,1.008,[],https://twitter.com/Tino_Chrupalla/status/1471...,61804,AfD,1,gibt konservative erneuerung steht ausgrenzung...
58862,58862,Tino Chrupalla,Tino_Chrupalla,2021-12-19 09:27:23,Tino Chrupalla\n@Tino_Chrupalla\n·\n19. Dez. 2021,Wir wünschen Ihnen und Ihrer Familie einen bes...,NaN,449.0,346.0,2.648,['https://pbs.twimg.com/media/FG9dwrcXIAUF5BP?...,https://twitter.com/Tino_Chrupalla/status/1472...,61805,AfD,0,wünschen familie besinnlichen advent


## 4. Code

In [5]:
text_data = []
for text in df_mentions['clean_text'].astype(str):
    tokens = word_tokenize(text)
    text_data.append(tokens)

In [6]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

In [7]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=N_TOPICS, passes=20, alpha='auto', per_word_topics=True)

In [8]:
lda_model.save("model/model_mentions.model")

In [9]:
topics = lda_model.show_topics(num_topics=N_TOPICS, num_words=20, formatted=False)
for topic in topics:
    topic_num = topic[0] + 1
    words = [word[0] for word in topic[1]]
    scores = [round(word[1], 4) for word in topic[1]]
    print(f"Topic #{topic_num}:")
    for i in range(len(words)):
        print(f"{words[i]} ({scores[i]}) ", end="")

Topic #1: genau, endlich, ach, nix, vielen, bundeswehr, weidel, dank, zurück, deutlich, mensch, rente, etc, fckafd, bringt, gelesen, fordern, sozial, gott, verdient
Topic #2: geld, meinung, thema, finde, brauchen, staat, aussage, steuern, punkt, beispiel, oft, funktioniert, offensichtlich, afghanistan, euro, wirtschaft, sprechen, ganzen, kosten, seite
Topic #3: ja, mal, schon, afd, linke, immer, partei, gibt, wäre, menschen, wer, geht, gut, einfach, warum, deutschland, ganz, macht, amp, cdu
Topic #4: wagenknecht, sagt, impfung, bundestag, falsch, noafd, grüne, sozialismus, völlig, kaum, darauf, verstanden, politische, probleme, usw, fordert, sache, bisschen, erstmal, ziel
Topic #5: herr, lieber, ddr, stimmt, merkel, gewählt, hätten, stimmen, wort, volk, fehler, erfolg, stellt, klimaschutz, glückwunsch, schauen, herrn, niemals, amt, früher
Topic #6: lassen, recht, impfen, bekommen, halten, tweet, verstehen, wieso, nazis, wichtig, heißt, passt, peinlich, finden, kannst, covid, spricht, w

## 5. Topics

In [10]:
topic_tokens = [word[0] for word in topics[8][1]]

In [11]:
df_mentions['date'] = pd.to_datetime(df_mentions['date'])

In [12]:
for token in topic_tokens:
    df_mentions[token] = 0

for index, row in df_mentions.iterrows():
    month = row['date'].month
    for token in topic_tokens:
        if token in row['clean_text']:
            df_mentions.at[index, token] += 1

TypeError: argument of type 'float' is not iterable

In [ ]:
grouped_mentions_df = df_mentions.groupby(df_mentions['date'].dt.month).sum()

In [ ]:
for token in topic_tokens:
    total_count = grouped_mentions_df[token].sum()
    grouped_mentions_df[token] = grouped_mentions_df[token] / total_count

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
sns.heatmap(grouped_mentions_df[topic_tokens].transpose(), cmap='Blues')
plt.title('Topic 1 Token Heatmap')
plt.xlabel('Month')
plt.ylabel('Token')
plt.show()

In [ ]:
grouped_mentions_df[topic_tokens].transpose()

In [ ]:
df_mentions[df_mentions['text'].str.contains('katjakipping')]